In [ ]:
pip install pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Assignment").getOrCreate()

In [5]:
df = spark.read.csv('/content/dataset', header=True, inferSchema=True)
df.show(10)

+-------+--------------------+-----+----------+-----------------+----------+---------------+
|   Nama|               Email| Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+-------+--------------------+-----+----------+-----------------+----------+---------------+
|Anthony|   dawnday@gmail.com| 24.0|   3553265|       01/13/1974|    Laptop|        2736049|
| Robert|    pbrown@gmail.com| NULL|   5525753|       03-05-2004|    Tablet|        4996241|
|Melissa| ethan99@clayton.com| NULL|   4911936|       28-04-1970|  Handpone|        3169456|
|Timothy|johnsonmegan@gmai...|200.0|3140485039|       03/09/2003|    Laptop|         506622|
|  Kelly|keithwebster(at)e...|200.0|3833611071|       17-01-2022|   Tablett|        9969204|
|  Jared| qwiggins@hodges.com| 69.0|   8024480|       02-01-2005|   Tablett|        3093351|
| Nicole|ogonzalez@hotmail...|200.0|4286346096|       08-01-2024| Handphone|        3007158|
|  Mason|lisasmith@hotmail...| NULL|   9858877|       10/12/1994|Smart

In [8]:
df.printSchema()
df.describe().show()
df.columns

root
 |-- Nama: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Umur: double (nullable = true)
 |-- Pendapatan: long (nullable = true)
 |-- Tanggal Pembelian: string (nullable = true)
 |-- Produk: string (nullable = true)
 |-- Nilai Transaksi: integer (nullable = true)

+-------+------+-------------------+------------------+--------------------+-----------------+---------+------------------+
|summary|  Nama|              Email|              Umur|          Pendapatan|Tanggal Pembelian|   Produk|   Nilai Transaksi|
+-------+------+-------------------+------------------+--------------------+-----------------+---------+------------------+
|  count|   100|                100|                68|                 100|              100|      100|               100|
|   mean|  NULL|               NULL|111.86764705882354|     1.65132351882E9|             NULL|     NULL|        5556026.09|
| stddev|  NULL|               NULL| 77.42110175763524|1.8103144429023266E9|             N

['Nama',
 'Email',
 'Umur',
 'Pendapatan',
 'Tanggal Pembelian',
 'Produk',
 'Nilai Transaksi']

In [9]:
from pyspark.sql.functions import col

df = spark.read.csv('/content/dataset', header=True, inferSchema=True)
df = df.dropDuplicates()
df = df.na.drop(subset=["nama"])
df = df.fillna({'email': 'Unknown'})
df = df.withColumn('umur', col('umur').cast('int'))
df.show(100)

+----------+--------------------+----+----------+-----------------+----------+---------------+
|      Nama|               Email|umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+----------+--------------------+----+----------+-----------------+----------+---------------+
|   Timothy|johnsonmegan@gmai...| 200|3140485039|       03/09/2003|    Laptop|         506622|
|      Mike|owensrobert@hotma...|  59|3741985672|       11/28/2004|    Laptop|        7711290|
|  Madeline|martinezmario@gma...| 200|3200189219|       12/05/1979|Smartwatch|        1908029|
|   Raymond|santiagoscott(at)...| 200|2866365700|       07/18/1993|Smartwatch|        6530558|
|  Benjamin|brittanygreen@mar...| 200|4075982395|       17-02-1982|    Tablet|        9067792|
|    Leslie|maysannette@gmail...| 200|   8372468|       09/13/1981|Smartwatch|        4957082|
|   Cynthia|lonnieandrews@cis...|  49|   5139026|       02/10/2000|   Tablett|        2393089|
|    Debbie| allison19@yahoo.com|  41|4974434132| 

In [12]:
from pyspark.sql.functions import col

df = df.withColumn("Umur", col("Umur").cast("int"))
df = df.filter(col('Umur') !=200)
df = df.fillna({"Umur": 0})

df.show()

+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Nama|               Email|Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Mike|owensrobert@hotma...|  59|3741985672|       11/28/2004|    Laptop|        7711290|
|  Cynthia|lonnieandrews@cis...|  49|   5139026|       02/10/2000|   Tablett|        2393089|
|   Debbie| allison19@yahoo.com|  41|4974434132|       24-04-1982|Headphonee|        7488725|
|   Rachel|jjenkins@osborne....|  66|   6422484|       02-08-1993| Smartwach|        5974942|
|     Sean|ronald86(at)email...|  43|   7309258|       09/21/2020|    Laptop|        6997815|
|   Amanda|melissa50@hotmail...|  45|   4765981|       13-06-2014|Smartwatch|        2110772|
|   Robert| jason16@hotmail.com|  31|   6167831|       05/31/1972| Handphone|        4166003|
|    James|angela86@hernande...|  21|1750047896|       07/07

In [13]:
from pyspark.sql.functions import col, when, to_date

df = df.withColumn("Tanggal Pembelian",
    when(col("Tanggal Pembelian").rlike(r"\d{2}/\d{2}/\d{4}"),
         to_date(col("Tanggal Pembelian"), "MM/dd/yyyy"))
    .otherwise(to_date(col("Tanggal Pembelian"), "dd-MM-yyyy"))
)

df.show()

+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Nama|               Email|Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Mike|owensrobert@hotma...|  59|3741985672|       2004-11-28|    Laptop|        7711290|
|  Cynthia|lonnieandrews@cis...|  49|   5139026|       2000-02-10|   Tablett|        2393089|
|   Debbie| allison19@yahoo.com|  41|4974434132|       1982-04-24|Headphonee|        7488725|
|   Rachel|jjenkins@osborne....|  66|   6422484|       1993-08-02| Smartwach|        5974942|
|     Sean|ronald86(at)email...|  43|   7309258|       2020-09-21|    Laptop|        6997815|
|   Amanda|melissa50@hotmail...|  45|   4765981|       2014-06-13|Smartwatch|        2110772|
|   Robert| jason16@hotmail.com|  31|   6167831|       1972-05-31| Handphone|        4166003|
|    James|angela86@hernande...|  21|1750047896|       1970-

In [15]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("Produk",
    regexp_replace("Produk", "Handpone", "Handphone"))
df = df.withColumn("Produk",
    regexp_replace("Produk", "Tablett", "Tablet"))
df = df.withColumn("Produk",
    regexp_replace("Produk", "Headphonee", "Headphone"))

df.show()

+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Nama|               Email|Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Mike|owensrobert@hotma...|  59|3741985672|       2004-11-28|    Laptop|        7711290|
|  Cynthia|lonnieandrews@cis...|  49|   5139026|       2000-02-10|    Tablet|        2393089|
|   Debbie| allison19@yahoo.com|  41|4974434132|       1982-04-24| Headphone|        7488725|
|   Rachel|jjenkins@osborne....|  66|   6422484|       1993-08-02| Smartwach|        5974942|
|     Sean|ronald86(at)email...|  43|   7309258|       2020-09-21|    Laptop|        6997815|
|   Amanda|melissa50@hotmail...|  45|   4765981|       2014-06-13|Smartwatch|        2110772|
|   Robert| jason16@hotmail.com|  31|   6167831|       1972-05-31| Handphone|        4166003|
|    James|angela86@hernande...|  21|1750047896|       1970-

In [24]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("Email", regexp_replace("Email", r"\s*at\s*|\s+at\s+|at", "@"))

df.show()

+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Nama|               Email|Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Mike|owensrobert@hotma...|  59|3741985672|       2004-11-28|    Laptop|        7711290|
|  Cynthia|lonnieandrews@cis...|  49|   5139026|       2000-02-10|    Tablet|        2393089|
|   Debbie| allison19@yahoo.com|  41|4974434132|       1982-04-24| Headphone|        7488725|
|   Rachel|jjenkins@osborne....|  66|   6422484|       1993-08-02| Smartwach|        5974942|
|     Sean|ronald86(@)email.com|  43|   7309258|       2020-09-21|    Laptop|        6997815|
|   Amanda|melissa50@hotmail...|  45|   4765981|       2014-06-13|Smartwatch|        2110772|
|   Robert| jason16@hotmail.com|  31|   6167831|       1972-05-31| Handphone|        4166003|
|    James|angela86@hernande...|  21|1750047896|       1970-

In [28]:
df.write.csv('data_transaksi_clean.csv', header=True, mode='overwrite')
df.show(100)

+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Nama|               Email|Umur|Pendapatan|Tanggal Pembelian|    Produk|Nilai Transaksi|
+---------+--------------------+----+----------+-----------------+----------+---------------+
|     Mike|owensrobert@hotma...|  59|3741985672|       2004-11-28|    Laptop|        7711290|
|  Cynthia|lonnieandrews@cis...|  49|   5139026|       2000-02-10|    Tablet|        2393089|
|   Debbie| allison19@yahoo.com|  41|4974434132|       1982-04-24| Headphone|        7488725|
|   Rachel|jjenkins@osborne....|  66|   6422484|       1993-08-02| Smartwach|        5974942|
|     Sean|ronald86(@)email.com|  43|   7309258|       2020-09-21|    Laptop|        6997815|
|   Amanda|melissa50@hotmail...|  45|   4765981|       2014-06-13|Smartwatch|        2110772|
|   Robert| jason16@hotmail.com|  31|   6167831|       1972-05-31| Handphone|        4166003|
|    James|angela86@hernande...|  21|1750047896|       1970-